# Section 2-3 - Convolutional Neural Network

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('../data/mnist.csv')

In [2]:
df_train = df.iloc[:33600, :]

X_train = df_train.iloc[:, 1:].values / 255.
y_train = df_train['label'].values
y_train_onehot = pd.get_dummies(df_train['label']).values

In [3]:
df_test = df.iloc[33600:, :]

X_test = df_test.iloc[:, 1:].values / 255.
y_test = df_test['label'].values

## Benchmark

In [4]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, df_train['label'].values)

y_prediction = model.predict(X_test)
print "\naccuracy", np.sum(y_prediction == df_test['label'].values) / float(len(y_test))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

accuracy 0.937857142857


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


## CNN

In [5]:
# https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Convolution2D, MaxPooling2D, Flatten

start = time()

img_rows, img_cols = 28, 28
nb_filters = 32
pool_size = (2, 2)
kernel_size = (3, 3)

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print '\ntime taken %s seconds' % str(time() - start)

Using TensorFlow backend.


Epoch 1/10
33600/33600 [==============================] - 46s - loss: 0.3342 - acc: 0.8968    
Epoch 2/10
33600/33600 [==============================] - 46s - loss: 0.1438 - acc: 0.9573    
Epoch 3/10
33600/33600 [==============================] - 47s - loss: 0.1091 - acc: 0.9682    
Epoch 4/10
33600/33600 [==============================] - 45s - loss: 0.0914 - acc: 0.9735    
Epoch 5/10
33600/33600 [==============================] - 46s - loss: 0.0813 - acc: 0.9755    
Epoch 6/10
33600/33600 [==============================] - 48s - loss: 0.0727 - acc: 0.9788    
Epoch 7/10
33600/33600 [==============================] - 46s - loss: 0.0676 - acc: 0.9801    
Epoch 8/10
33600/33600 [==============================] - 47s - loss: 0.0624 - acc: 0.9818    
Epoch 9/10
33600/33600 [==============================] - 43s - loss: 0.0577 - acc: 0.9829    
Epoch 10/10
33600/33600 [==============================] - 44s - loss: 0.0567 - acc: 0.9833    

time taken 464.216364145 seconds


In [6]:
y_prediction = model.predict_classes(X_test)
print "\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test))

8400/8400 [==============================] - 4s     

accuracy 0.985238095238
